In [8]:
import base

import pandas as pd

import fund_scanner.database



In [5]:
df = pd.read_csv('http://quotes.money.163.com/service/chddata.html?code=0000001&start=19901219&end=20170727&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;VOTURNOVER;VATURNOVER', encoding = "gb2312")


In [9]:
df = df.sort_values(by='日期')
df.head(2)


,日期,股票代码,名称,收盘价,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,成交量,成交金额
6506,1990-12-19,'000001,上证指数,99.98,99.98,95.79,96.05,None,None,None,1260,494000.0
6505,1990-12-20,'000001,上证指数,104.39,104.39,99.98,104.30,99.98,4.41,4.4109,197,84000.0


In [10]:
benchmarks_id = 1

with fund_scanner.database.get_connection() as cursor:
    for index, row in df.iterrows():
        #print(row['净值日期'])
        sql = 'INSERT ignore INTO `benchmarks_historical_price` '+\
        '(`benchmarks_id`, `benchmarks_price_date`, `benchmarks_price`, `benchmarks_price_adjust`) '+\
        'VALUES (%s,%s,%s,%s)'
        cursor.execute(sql, (benchmarks_id, row['日期'], 
                             base.extract_float(row, '收盘价'), base.extract_float(row, '开盘价')))

        sql = 'Update `benchmarks` set `historical_price_time`=Now() where `benchmarks_id`=%s'
        cursor.execute(sql, (benchmarks_id))
